# Экпериментальный метод аугментации данных при помощи модели Stable Diffusion
# (Еще не тестил)

Предварительно в label-studio разметили области в которых будем генерировать дефекты

In [1]:
import json

In [2]:
ann = json.load(open('/app/img/result.json'))

FileNotFoundError: [Errno 2] No such file or directory: '/app/img/result.json'

In [ ]:
categories_and_imgs_for_aug = {
    "0" : [],
    "1" : [],
    "2" : [],
    "3" : [],
    "4" : [],
}


for ann_image in ann['images']:
    img_id = ann_image['id']
    img_name = ann_image['file_name']
    
    for ann_ann in ann['annotations']:
        if ann_ann['image_id'] == img_id:
            category_id = ann_ann['category_id']
            categories_and_imgs_for_aug[str(category_id)].append(
                {
                    'bbox':ann_ann['bbox'],
                    'file_name': img_name,
                }
            )

            
categories_and_imgs_for_aug

In [ ]:
import os
import math

from PIL import Image, ImageDraw
from chitra.image import Chitra


aug_ann = dict()
aug_ann['categories'] = ann['categories']
aug_ann['info'] = ann['info']
aug_ann['images'] = []
aug_ann['annotations'] = []



prompts_for_categories = {
    "0" : 'fistual on a metal pipe', #Выход трубы из ряда
    "1" : 'black corrosion crack on old metal pipe', #Крип
    "2" : 'a long black crack on an old pipe located vertically', #Разрыв
    "3" : 'black corrosion crack on old metal pipe', #Свищ
    "4" : 'a long black crack on an old pipe located vertically', #Трещина
}


i=0
image_id = 0
annotaion_id = 0
for category_id, annotations in categories_and_imgs_for_aug.items():
    if(len(annotations) != 0):
        for annotation in annotations:
            print(annotations)

            img_path = annotation['file_name']
            img_name = os.path.basename(img_path)
            img_name_without_ext = img_name.split('.')[0]
            path_to_save_cropped_img = f"/app/img/cropped/{img_name_without_ext}_crop_category{str(category_id)}_{i}.png"
            img = Image.open(img_path).convert('RGBA')
            original_size = img.size
            # img = img.resize((1380,800))
            draw = ImageDraw.Draw(img)
            bbox = annotation['bbox']
            x0, y0 = (bbox[0], bbox[1])
            x1, y1 = (x0 + bbox[2], y0 + bbox[3])
            draw.rectangle([x0, y0, x1, y1], fill=(0,0,0,0)) #[(x0, y0), (x1, y1)]
            img = img.resize((1380,800))
            img.save(path_to_save_cropped_img)
            
            for j in range(100):
                path_to_save_auged_img = f"/app/img/augs/{img_name_without_ext}_aug_category{str(category_id)}_{i}_{j}.png"
                img = Image.open(path_to_save_cropped_img)
                print(f"predicted img: {path_to_save_cropped_img}")
                w,h = img.size
                print(f"positive prompt: {prompts_for_categories[str(category_id)]}")
                new_img = run_outpaint(
                    img=img,
                    prompt_text = prompts_for_categories[str(category_id)],
                    negative_prompt_text='',
                    strength=1,
                    guidance=16,
                    step=20,
                    resize_check=True,
                    fill_mode="patchmatch",
                    enable_safety=False,
                    w=w,
                    h=h
                )
                print(f"img.size={img.size}")
                print(f"new_img.size={new_img.size}")
                new_img.save(path_to_save_auged_img)

                print("FOR COCO-------------------------")

                chitra_image = Chitra(img_path, [x0, y0, x1, y1], 'pipe')
                chitra_image.resize_image_with_bbox((1380,800))
                chitra_bbox = chitra_image.bboxes[0]
                new_x0, new_y0, new_x1, new_y1 = (chitra_bbox.x1, chitra_bbox.y1, chitra_bbox.x2, chitra_bbox.y2)
                new_bbox = [new_x0, new_y0, new_x1 - new_x0, new_y1 - new_y0]

                aug_ann['images'].append(
                    {
                        "width": w,
                        "height": h,
                        "id": image_id,
                        "file_name": os.path.basename(path_to_save_auged_img)
                    }
                )

                aug_ann['annotations'].append(
                    {
                        "id": annotaion_id,
                        "image_id": image_id,
                        "category_id": int(category_id),
                        "segmentation": [],
                        "bbox": [
                          int(new_bbox[0]),
                          int(new_bbox[1]),
                          math.ceil(new_bbox[2]),
                          math.ceil(new_bbox[3])
                        ],
                        "ignore": 0,
                        "iscrowd": 0,
                        "area": math.ceil(new_bbox[2]) * math.ceil(new_bbox[3])
                    }
                )

                i += 1
                image_id += 1
                annotaion_id += 1
            


Сохраняем аннотации в формате coco:

In [ ]:
with open("../img/test_stable_aug_ann_with_three_imgs.json", 'w') as f:
    json.dump(aug_ann, f)

### COCO to label-studio format

In [ ]:
!git clone https://github.com/heartexlabs/label-studio-converter.git

In [ ]:
pip install -e ./label-studio-converter

In [ ]:
!label-studio-converter import coco -i img/test_stable_aug_ann_with_three_imgs.json -o img/stable_aug_for_label_studio.json